In [1]:
from tensorflow import keras
import time
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, BatchNormalization, AveragePooling2D
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
import tensorflow as tf
import logging
import numpy as np


In [2]:
def make_cnn():
   img_size = 28 # размер картинки
   num_channels = 1  # Для черно-белых изображений
   output = 10  # количество выходных нейронов
   neural_network = Sequential() # Создание нейросети
    
    # добавление сверточных слоев
    
   neural_network.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu',
            padding='same', 
input_shape=(img_size, img_size, num_channels)))
   neural_network.add(MaxPooling2D(pool_size=(2, 2)))
   neural_network.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu',
            padding='same'))
   neural_network.add(MaxPooling2D(pool_size=(2, 2)))
   neural_network.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu',
            padding='same'))
   neural_network.add(MaxPooling2D(pool_size=(2, 2)))
   neural_network.add(Flatten())

   # Добавление обычного слоя нейронов
   neural_network.add(Dense(128, activation='relu'))
   # Добавление выходного слоя
   neural_network.add(Dense(output, activation='softmax'))
   neural_network.compile(optimizer=Adam(), loss='categorical_crossentropy',
            metrics=['accuracy'])
    
   return neural_network

In [3]:
def train_cnn(neural_network):
   (train_digits, train_labels), (test_digits, test_labels) = keras.datasets.mnist.load_data() 

   img_size = 28 # размер картинки
   num_channels = 1  # Для черно-белых изображений

   # преобразование полученного изображение в 28*28
   train_data = np.reshape(train_digits, (train_digits.shape[0],img_size, img_size, num_channels))
   train_data = train_data.astype('float32') / 255.0
   output = 10 # Выходные слои 
   train_labels_cat = keras.utils.to_categorical(train_labels, output)

   # преобразование размера и цвета
   val_data = np.reshape(test_digits, (test_digits.shape[0], img_size, img_size, num_channels))
   val_data = val_data.astype('float32') / 255.0
   # encode the labels - we have 10 output classes
   val_labels_cat = keras.utils.to_categorical(test_labels, output)

   print("Обучение нейросети...")
   time.sleep(0.5)
   t_start = time.time() # начало отчета

   # Обучение нейросети
   neural_network.fit(train_data, train_labels_cat, epochs=8, batch_size=64,
        validation_data=(val_data, val_labels_cat))

   print("Выполнено за:", time.time() - t_start) # 

   return neural_network

In [ ]:
model = make_cnn()
train_cnn(model)
model.save('cnn_digits.h5')

In [ ]:
def cnn_digits_predict(model, image_file): #прогноз нейростеи
   img_size = 28 # размер картинки
   img = keras.preprocessing.image.load_img(image_file, 
target_size=(img_size, img_size), color_mode='grayscale') # получение картинки
   img_arr = np.expand_dims(img, axis=0)
   img_arr = 1 - img_arr/255.0
   img_arr = img_arr.reshape((1, 28, 28, 1))

   result = model.predict_classes([img_arr]) # результат работы
   return result[0] # возврат предсказываемого значения

In [ ]:
from tkinter import Tk     #Для интерфейса
from tkinter.filedialog import askopenfilename #Для запроса файла у пользователя
from PIL import Image #Для изображений
tk = Tk()
tk.withdraw() # для запуска библиотеки tkinter
file = askopenfilename() # для открытия файла

model = tf.keras.models.load_model('cnn_digits.h5') #загрузка сохраненной модели нейросети
print(cnn_digits_predict(model, file)) #вывод результата прогноза нейросети
